### Code done by Serden-Yilmaz Kose, Jesper Nyman and Jussi Saariniemi

In [1]:
import nltk
import numpy as np
from nltk.corpus import genesis
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
"""
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('genesis')

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
"""

"\nnltk.download('stopwords')\nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('genesis')\n\nnltk.download('punkt_tab')\nnltk.download('averaged_perceptron_tagger_eng')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('vader_lexicon')\n"

In [3]:
import pandas as pd

path = "./kurrek.2020.slur-corpus.csv"
data = pd.read_csv(path, sep=",", header = 0, on_bad_lines='skip')

In [4]:
# Make function to check if string contains any element from a string
# https://bobbyhadz.com/blog/python-check-if-string-contains-element-from-list
def is_element_in_string(string, flist):
    if any((match := substring) in string for substring in flist):
        print('The string contains at least one element from the list')
    return match

In [5]:
# Replace slur with [REDACTED], and print
# Make new data list using pandas
# This one is only to print the strings, to avoid printing harmful language. 

path = "./kurrek.2020.slur-corpus.csv"
redacted_data = pd.read_csv(path, sep=",", header = 0, on_bad_lines='skip')

# Iterate for each index in redacted_data, and modify the 'body' to replace the 
# slur with [REDACTED]. The indexes will remain the same as the one found in the original
# document, so this is really to just print out certain rows whilst avoiding printing out the
# harmful language (the slurs).
for index, row in redacted_data.iterrows():
    T1, T2 = str(row['body']), str(row['slur'])
    redacted_data.at[index, 'body'] = T1.lower().replace(T2, "[REDACTED]")

## Task 1

In [6]:
"""By constructing a dataframe of posts assigned to the same category, suggest a script that outputs the
vocabulary set of each category, the size of the vocabulary, the total number of tokens, the average
number of tokens per post and its standard deviation, the average number of pronouns per post and the
associated standard deviation, the ten most frequent tokens in each category, excluding the stopword list.
Represent the statistical result in a clear table and discuss whether some parameters are most relevant to
discriminate a given category."""

'By constructing a dataframe of posts assigned to the same category, suggest a script that outputs the\nvocabulary set of each category, the size of the vocabulary, the total number of tokens, the average\nnumber of tokens per post and its standard deviation, the average number of pronouns per post and the\nassociated standard deviation, the ten most frequent tokens in each category, excluding the stopword list.\nRepresent the statistical result in a clear table and discuss whether some parameters are most relevant to\ndiscriminate a given category.'

In [7]:
# The following is a list of categories, and the number of frequencies
"""
DEG 	Derogatory 	20531
NDG 	Non Derogatory Non Appropriative 	16729
HOM 	Homonym 	1998
APR 	Appropriative 	553
CMP 	Noise 	189
"""
# Create dictionary variables to store data stored about each category
def dict_template():
    temp_dict = {"vocab_set"     : [],
                 "vocab_size"    : 0,
                 "total_tokens"  : 0,
                 "avg_tokens"    : 0,
                 "tokens_standev": 0,
                 "avg_pronouns"  : 0,
                 "prp_standev"   : 0,
                 "freq_words"    : [],
                 "all_words"     : [],
                 "entire_words"  : [] 
           }
    return temp_dict

deg_dict = dict_template()
ndg_dict = dict_template()
hom_dict = dict_template()
apr_dict = dict_template()
cmp_dict = dict_template()

# Make category dictionary to store dictionaries by index
categ_dict = {0: deg_dict,
              1: ndg_dict,
              2: hom_dict,
              3: apr_dict,
              4: cmp_dict
             }

# Store names of categories by index in this dictionary
categ_string_list = {0: "DEG",
                     1: "NDG",
                     2: "HOM",
                     3: "APR",
                     4: "CMP"
             }

# Create dataframe of posts within the same category
bodies_dataframe = {"DEG": [],
                    "NDG": [],
                    "HOM": [],
                    "APR": [],
                    "CMP": []
                   }

# Iterate through each entry of data, and append the body to its respected category
for index, row in data.iterrows():
    # Create body and label integer and string
    body, label = str(row['body']), str(row['gold_label'])
    if label in categ_string_list.values():
        bodies_dataframe[label].append(body)
# Convert to dataframe with two columns
bodies_dataframe = pd.DataFrame(bodies_dataframe.items(), columns=['Category', 'Bodies'])

In [8]:
# Function to find the vocabulary set per category 
# Flatten out list of lists for vocab set
# https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists
# https://stackoverflow.com/questions/10677020/real-word-count-in-nltk
# https://www.geeksforgeeks.org/python-statistics-stdev/
# https://stackoverflow.com/questions/35086440/python-how-to-compute-the-top-x-most-frequently-used-words-in-an-nltk-corpus
import itertools
import statistics
from nltk import FreqDist
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Create stopwords list, and add words that do not belong in the vocabulary 
Stopwords = set(upper_word.lower() for upper_word in nltk.corpus.stopwords.words('english'))
Stopwords.add("http")
Stopwords.add("https")
Stopwords.add("com")
Stopwords.add("www")
Stopwords.add("r")
Stopwords.add("html")

In [9]:
# Make function to go through ALL documents under a specific category
def find_vocab_set(categ):
    # Create temporary variables and lists to store analyzed data
    tokenized_bodies = []
    total_tokens = 0
    total_bodies = 0
    size_of_doc = []
    pronouns_num_list = []
    total_pronouns = 0
    most_frequent_words = []
    # Iterate through each body in the dataframe, under the specified category
    for body in bodies_dataframe[bodies_dataframe['Category'] == categ_string_list[categ]]['Bodies'].sum():
        # Tokenize the body and turn it into lowercase
        tokenized_body = tokenizer.tokenize(body.lower())
        # Print tokenized body
        tokenized_bodies.extend(tokenized_body)
        # Update total tokens
        total_tokens += len(tokenized_body)
        # Update size of docs list
        size_of_doc.append(len(tokenized_body))
        # Update pronounds list
        tmp_pronoun = extract_pronouns(body)
        pronouns_num_list.append(tmp_pronoun)
        # Update total pronounds count
        total_pronouns += tmp_pronoun
        # Update total bodies
        total_bodies += 1
    # Remove doubles
    categ_dict[categ]["vocab_set"] = list(set(tokenized_bodies))
    # Update vocab size
    categ_dict[categ]["vocab_size"] = len(categ_dict[categ]["vocab_set"])
    # Update total number of tokens
    categ_dict[categ]["total_tokens"] = total_tokens
    # Update avg tokens:
    categ_dict[categ]["avg_tokens"] = total_tokens / total_bodies
    # Update token standard deviation
    categ_dict[categ]["tokens_standev"] = statistics.stdev(size_of_doc)
    # Update avg pronouns
    categ_dict[categ]["avg_pronouns"] = total_pronouns / total_bodies
    # Update pronoun standard deviation
    categ_dict[categ]["prp_standev"] = statistics.stdev(pronouns_num_list)
    # Remove stopwords
    bodies_without_stopwords = [word.lower() for word in tokenized_bodies if word.isalpha() and word.lower() not in Stopwords]
    # Update frequent tokens, without stopwords
    most_frequent_words = FreqDist(bodies_without_stopwords)
    categ_dict[categ]["freq_words"] = most_frequent_words.most_common(10)
    categ_dict[categ]["all_words"] = most_frequent_words.most_common(categ_dict[categ]["vocab_size"])
    categ_dict[categ]["entire_words"] = FreqDist(tokenized_bodies).most_common(categ_dict[categ]["vocab_size"])

# Function to extract number of pronouns in a given string
def extract_pronouns(string):
    # Extract pronouns using pos_tag
    pos_list = nltk.pos_tag(word_tokenize(string))
    number_of_pronouns = 0
    # Run through every entry
    for pos in pos_list:
        if pos[1] == "PRP":
            number_of_pronouns += 1
    return number_of_pronouns

In [10]:
find_vocab_set(0)
find_vocab_set(1)
find_vocab_set(2)
find_vocab_set(3)
find_vocab_set(4)

In [11]:
# Print DataFrame without array values (Since they have more than one element)
# https://stackoverflow.com/questions/31433989/return-copy-of-dictionary-excluding-specified-keys

DataFrame_dict = {}

def without_keys(d, keys):
    return {k: v for k, v in d.items() if k not in keys}

for key, value in categ_dict.items():
    """
    print(f"Info regarding category    {categ_string_list[key]}:")
    print(f"Vocab size:                {value["vocab_size"]}")
    print(f"Total size:                {value["total_tokens"]}")
    print(f"Average tokens:            {value["avg_tokens"]["num_tokens"]:.2f}")
    print(f"Token standard dev:        {value["avg_tokens"]["standard_dev"]:.2f}")
    print(f"Average pronouns:          {value["avg_pronouns"]["num_tokens"]:.2f}")
    print(f"Pronoun standard dev:      {value["avg_pronouns"]["standard_dev"]:.2f}")
    print(f"Ten frequent words:        {value["freq_words"]}")
    print("\n")
    """
    DataFrame_dict[categ_string_list[key]] = without_keys(value, ["vocab_set", "freq_words","all_words", "entire_words"])
# Print dataframe using Pandas
print(pd.DataFrame(DataFrame_dict))

In [12]:
# Print top 10 frequent words
for key, value in categ_dict.items():
    print(f"Top ten most frequent tokens for categ {categ_string_list[key]}: \n{value["freq_words"]}")

In [13]:
"""WE have gathered lots of data, and can notice interesting aspects of our findings. First of all, the DEG category
has the most words by far, followed, by NDG, HOM, APR, and finally CMP. However, HOM and NDG have more tokens per
document than the other categories. The lowest standard deviation for the tokens was found in the CMP and DEG 
category. The most pronouns per token were found in APR and NDG. It's respective lowest standardized deviation was 
found in the CMP and DEG categories."""

"WE have gathered lots of data, and can notice interesting aspects of our findings. First of all, the DEG category\nhas the most words by far, followed, by NDG, HOM, APR, and finally CMP. However, HOM and NDG have more tokens per\ndocument than the other categories. The lowest standard deviation for the tokens was found in the CMP and DEG \ncategory. The most pronouns per token were found in APR and NDG. It's respective lowest standardized deviation was \nfound in the CMP and DEG categories."

## Task 2

In [14]:
"""Suggest a script to draw and evaluate the zipf’s law fitting for the dataframe of each category, and
computing the corresponding R2 and adjusted R2 statistics."""

'Suggest a script to draw and evaluate the zipf’s law fitting for the dataframe of each category, and\ncomputing the corresponding R2 and adjusted R2 statistics.'

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from nltk import FreqDist
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
# https://www.geeksforgeeks.org/zipfs-law/#what-is-zipfs-law 
# https://www.geeksforgeeks.org/r-squared-vs-adjusted-r-squared-difference/
# https://www.statology.org/r-squared-in-python/
# https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
# https://www.statology.org/adjusted-r-squared-in-python/

# Make dict to determine if function handles entire words, or all words(without stopwords)
freq_type_dict = {0: "entire_words",
                  1: "all_words"}

# Make function to plot Zipf's law fitting and find R2 and R2 adjusted 
def zipfs_law_fitting_nltk(categ_dict, corpus_type):
    # Print the type of data we are using, stopwords or no stopwords
    if corpus_type == 0:
        print("FOLLOWING DATA INCLUDES STOPWORDS!!!\n")
    else:
        print("FOLLOWING DATA EXCLUDES STOPWORDS!!!\n")
    # Iterate through all entries in the category
    for key, value in categ_dict.items():
        # Get the tokens from the dictionary
        entire_words = value[freq_type_dict[corpus_type]]
        
        #Define two lists, rank and frequency
        rank = []
        frequency = []
        init = 0
        
        #Assign ranks based on frequencies of words
        for freq in entire_words:
            init+=1
            rank.append(init)
            frequency.append(freq[1])
        
        #Plot the rank and frequency
        plt.plot(np.log(rank),np.log(frequency))
        # Labelling the x axis
        plt.xlabel('Rank')
        # Labelling the y axis
        plt.ylabel('Frequency')
        
        # Providing a title to the graph
        plt.title(f"Zipf's law for {categ_string_list[key]}")

        # Getting r2 and adjusted r2 values
        #initiate linear regression model
        model = LinearRegression()

        X = np.array(rank).reshape(-1, 1)
        Y = frequency

        #fit regression model
        model.fit(X, Y)
        
        # Calculate R-squared of regression model
        log_ranks = np.log(rank)
        log_frequencies = np.log(frequency)
        slope, intercept, r_value, p_value, std_err = linregress(log_ranks, log_frequencies)
        r2 = r_value ** 2

        #display adjusted R-squared
        N = len(log_frequencies)
        r2_adjusted = 1 - ((1-r2)*(N - 1))/(N-1-1)
        
        # View R-squared value and adjusted
        print(f"R2 value for {categ_string_list[key]}         : {r2}")
        print(f"R2 adjusted value for {categ_string_list[key]}: {r2_adjusted}")


        # show linear regression
        log_ranks = np.log(rank)
        log_frequencies = np.log(frequency)
        slope, intercept, r_value, p_value, std_err = linregress(log_ranks, log_frequencies)
        plt.plot(log_ranks, slope * log_ranks + intercept, color='red', label=f'Fit: slope={slope:.2f}')
        
        # Show graph, after printing R2 and R2A 
        plt.show()

In [16]:
zipfs_law_fitting_nltk(categ_dict, 0)

In [17]:
"""We can see here that the Zipf's law fitting does apply to a large extent. Near the smaller ranks, it isn't as strong as it is 
near the end of the curve. However, I believe the reason behind that to be the excess use of particular tokens, specefically slurs.
That could be a possible reason behind this existing, although small, disparity from the expected red line."""

"We can see here that the Zipf's law fitting does apply to a large extent. Near the smaller ranks, it isn't as strong as it is \nnear the end of the curve. However, I believe the reason behind that to be the excess use of particular tokens, specefically slurs.\nThat could be a possible reason behind this existing, although small, disparity from the expected red line."

## Task 3

In [18]:
zipfs_law_fitting_nltk(categ_dict, 1)

In [19]:
""" Here we can also conclude that Zipf's law does apply to these documents. However, we must note that it isn't as strong
as in task 2. Since we are removing a lot of words, be ignoring the stopwords, this is a product of that. We can conclude that 
the presence of stopwords is quite important in applications of Zipf's Law. """

" Here we can also conclude that Zipf's law does apply to these documents. However, we must note that it isn't as strong\nas in task 2. Since we are removing a lot of words, be ignoring the stopwords, this is a product of that. We can conclude that \nthe presence of stopwords is quite important in applications of Zipf's Law. "

## Task 4

In [20]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

data = pd.DataFrame(data)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jussi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jussi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
# bodies contained values that were not strings, so we had to convert them to strings
data['body'] = data['body'].apply(lambda x: str(x) if not isinstance(x, str) else x)      

In [22]:
# tokenizing and lowering each body and then pos tagging them
pos_tagged = []
for body in data["body"]:
    tokens = word_tokenize(body.lower())
    tokens = [word for word in tokens if word.isalnum()]
    pos_tagged.append(nltk.pos_tag(tokens))

In [23]:
# counting the amount of each pos tag and sorting them from most frequent to least frequent
pos_amounts = {}
for pos in pos_tagged:
    for word, tag in pos:
        if tag in pos_amounts:
            pos_amounts[tag] += 1
        else:
            pos_amounts[tag] = 1

pos_amounts = dict(sorted(pos_amounts.items(), key=lambda item: item[1], reverse=True))

print(pos_amounts)

In [24]:
# Checking if Zipfs law distribution can be fitted for the frequencies of pos tags across all the bodies

from scipy.stats import linregress

frequencies = list(pos_amounts.values())
ranks = range(1, len(pos_amounts) + 1)

plt.figure(figsize=(10, 5))
plt.plot(ranks, frequencies, marker='o')
plt.xlabel('Rank')
plt.ylabel('Frequency')
plt.title('Rank/Frequency Plot')
plt.show()

log_ranks = np.log(ranks)
log_frequencies = np.log(frequencies)

plt.figure(figsize=(10, 5))
plt.plot(log_ranks, log_frequencies, marker='o')

slope, intercept, r_value, p_value, std_err = linregress(log_ranks, log_frequencies)
plt.plot(log_ranks, slope * log_ranks + intercept, color='red', label=f'Fit: slope={slope:.2f}')

plt.xlabel('Log(Rank)')
plt.ylabel('Log(Frequency)')
plt.title('Log-Log Plot')

plt.show()

print(f"R-squared: {r_value ** 2}")
print(f"Slope: {slope}")
print(f"P value: {p_value}")

print("There is some resemblance to Zipfs law, but not enough.")

## Task 5

In [25]:
# Checking if Heaps law can be fitted for each category of bodies

# Extracting the bodies sorted by category into a dataframe
bodies_by_category = {}

for index, row in data.iterrows():
    # Create body and label integer and string
    body, label = str(row['body']), str(row['gold_label'])
    if label in bodies_by_category:
        bodies_by_category[label].append(body)
    else:
        bodies_by_category[label] = [body]

bodies_by_category = pd.DataFrame(bodies_by_category.items(), columns=['Category', 'Bodies'])

# preprocessing the bodies and storing them in a new dataframe
tokens_by_category = {}

for index, row in bodies_by_category.iterrows():
    bodies, label = row['Bodies'], row['Category']

    for body in bodies:
        tokens = word_tokenize(body.lower())
        tokens = [word for word in tokens if word.isalnum()]
        if label in tokens_by_category:
            tokens_by_category[label].extend(tokens)
        else:
            tokens_by_category[label] = []
            tokens_by_category[label] = tokens

tokens_by_category = pd.DataFrame(tokens_by_category.items(), columns=['Category', 'Tokens'])

In [26]:
def DataPointsCalculator(category):
    tokens = tokens_by_category[tokens_by_category['Category'] == category]['Tokens'].sum()
    points = []
    unique_tokens = []

    for i in range(1, len(tokens)):
        if tokens[i] not in unique_tokens:
            unique_tokens.append(tokens[i])
        points.append((i, len(unique_tokens)))

    return points

uniqueTokensByTotalTokens = {
    'DEG': DataPointsCalculator('DEG'),
    'NDG': DataPointsCalculator('NDG'),
    'HOM': DataPointsCalculator('HOM'),
    'APR': DataPointsCalculator('APR'),
    'CMP': DataPointsCalculator('CMP')
}

In [27]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

def HeapsLaw(N, k, b):
    return k * N ** b

heaps_fitting_results = {}

for row, points in uniqueTokensByTotalTokens.items():
    x_values, y_values = zip(*points)
    x_values = np.array(x_values)
    y_values = np.array(y_values)

    popt, _ = curve_fit(HeapsLaw, x_values, y_values, maxfev=10000)
    k, b = popt
    
    y_pred = HeapsLaw(x_values, k, b)
    r2 = r2_score(y_values, y_pred)
    heaps_fitting_results[row] = {'k': k, 'beta': b, 'R2': r2}
    
    plt.plot(x_values, y_values, label='Data')
    plt.plot(x_values, y_pred, label=f'R^2={r2:.2f}, K={k:.2f}, beta={b:.2f}', linestyle='--')
    
    plt.xlabel('Total Tokens')
    plt.ylabel('Unique Tokens')
    plt.title(f'Heaps law fit for {row} category')
    plt.legend()
    plt.show()

    print(f"For {row} category with k={k:.2f} and b={b:.2f}, the R^2 value is {r2:.2f}")
    if r2 > 0.9:
        print("Heaps law fit is good")
    else:
        print("Heaps law fit is not good")
        


## Task 6

In [28]:
# Vocabulary for each category

vocabularies_by_category = {}

for index, row in tokens_by_category.iterrows():
    categ = row['Category']
    tokens = row['Tokens']
    vocabulary = set(tokens)
    vocabularies_by_category[categ] = vocabulary

In [29]:
categories = ['DEG', 'NDG', 'HOM', 'APR', 'CMP']

# Jaccard similarity between the vocabularies of categories
jaccard_similarities = {}
for i in range(0, len(categories)):
    for j in range(i + 1, len(categories)):
        category1 = categories[i]
        category2 = categories[j]
        vocabulary1 = vocabularies_by_category[category1]
        vocabulary2 = vocabularies_by_category[category2]
        jaccard_similarity = len(vocabulary1.intersection(vocabulary2)) / len(vocabulary1.union(vocabulary2))
        jaccard_similarities[(category1, category2)] = jaccard_similarity

print("Jaccard similarity between the vocabularies of each category pair:")
for categories, similarity in jaccard_similarities.items():
    print(f"{categories[0]} and {categories[1]}: {similarity:.2f}")


## Task 7

In [30]:
# Each body in tokenized form by category
token_bodies_by_category = {}

for index, row in bodies_by_category.iterrows():
    categ = row['Category']
    bodies = row['Bodies']
    for body in bodies:

        tokens = word_tokenize(body.lower())
        tokens = [word for word in tokens if word.isalnum()]

        if categ in token_bodies_by_category:
            token_bodies_by_category[categ].append(tokens)
        else:
            token_bodies_by_category[categ] = []
            token_bodies_by_category[categ].append(tokens)

In [31]:
# pos tagging
pos_token_bodies_by_category = {}

for row, token_bodies in token_bodies_by_category.items():
    pos_token_bodies_by_category[row] = []
    for token_body in token_bodies:
        pos_token_bodies_by_category[row].append(nltk.pos_tag(token_body))

In [32]:
# Finding the pos tags for tokens around the modal verb with window size of 2

pos_tags_around_MD = {}

for row, pos_token_bodies in pos_token_bodies_by_category.items():
    pos_tags_around_MD[row] = []
    for body in pos_token_bodies:
        for i in range(0, len(body)):
            if body[i][1] == 'MD':
                start = max(i - 2, 0)
                end = min(i + 3, len(body))
                window = [token[1] for j, token in enumerate(body[start:end]) if (start + j) != i]
                pos_tags_around_MD[row].append(window)

In [33]:
# Counting the frequencies for all the pos tags around the MD by category
# and saving them in a dataframe
pos_tags_around_MD_frequencies = {}

for row, pos_token_list in pos_tags_around_MD.items():
    pos_tags_around_MD_frequencies[row] = {}
    for pos_tag_list in pos_token_list:
        for pos_tag in pos_tag_list:
            if pos_tag in pos_tags_around_MD_frequencies[row]:
                pos_tags_around_MD_frequencies[row][pos_tag] += 1
            else:
                pos_tags_around_MD_frequencies[row][pos_tag] = 1

# Cleanup for the dataframe
pos_tags_around_MD_frequencies = pd.DataFrame(pos_tags_around_MD_frequencies)
pos_tags_around_MD_frequencies = pos_tags_around_MD_frequencies.dropna(axis=1, how='all')
pos_tags_around_MD_frequencies = pos_tags_around_MD_frequencies.fillna(0)
pos_tags_around_MD_frequencies = pos_tags_around_MD_frequencies.astype(int)

In [34]:
print(pos_tags_around_MD_frequencies)

In [35]:
# plotting the frequencies of pos tags around the MD with window of 2 for each category
for row in pos_tags_around_MD_frequencies.columns:
    plt.figure(figsize=(10, 6))
    
    sorted_values = pos_tags_around_MD_frequencies[row].sort_values(ascending=False)
    
    sorted_values.plot(kind='bar')
    plt.title(f'POS tag frequencies for tokens in window of 2 around modal verbs.\n For {row} category')
    plt.xlabel('POS Tag')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [36]:
# Cosine similarity between pos tag frequencies of each pair of categories

from sklearn.metrics.pairwise import cosine_similarity

category_vectors = pos_tags_around_MD_frequencies.T

cosine_sim_matrix = pd.DataFrame(cosine_similarity(category_vectors), index=category_vectors.index, 
                                 columns=category_vectors.index)
print("Cosine similarity matrix between the frequencies of pos tags in a window of 2 around modal verbs")
print("for each category pair:")
print(cosine_sim_matrix)

print("\nAll the categories are quite similar to each other according to cosine similarity.")
print("This way of comparing similarity does not take into account the sample sizes.")
print("Small sample size can lead to misleading results.")
print("Smallest sample is CMP with 133, and largest is NDG with 33729, ")
print("so the similarity between NDG and CMP is not as reliable as the similarity between NDG and other large set like DEG.")

## Task 8

In [ ]:
"""Use Vader sentiment analysis package https://github.com/cjhutto/vaderSentiment to computer the
overall sentiment of each post, then draw a 10-bin histogram showing the proportion of posts whose
overall sentiment score falls in a given bin. This yields four histogram distributions (one histogram
distribution for each dataframe). Next, use the standard Euclidean histogram distance to compute the
similarity between every pair of categories with respect to sentiment scores."""

In [ ]:
# Import sentece and initialize analyzer 
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Creat dictionary to store sentiment values as a element of a list, as a value
sentiment_dict  = {  "DEG" : [],
                     "NDG" : [],
                     "HOM" : [],
                     "APR" : [],
                     "CMP" : []
             }

In [ ]:
def find_vocab_sent(categ):
    for index, row in data.iterrows():
        # Create body and label integer and string
        body, label = str(row['body']), str(row['gold_label'])
        # Check if the label is deg, ndg, hom, apr, or cmp
        if label == categ_string_list[categ]:
            # Find sentiment score and append to respected category list
            vs = analyzer.polarity_scores(str(row['body']))
            sentiment_dict[categ_string_list[categ]].append(vs['compound'])
find_vocab_sent(0)
find_vocab_sent(1)
find_vocab_sent(2)
find_vocab_sent(3)
find_vocab_sent(4)

In [ ]:
import pprint
for key, value in sentiment_dict.items():
    print(f"Average sentiment for {key}: {sum(value) / len(value)}")

In [ ]:
# Plotting histograms
# https://www.geeksforgeeks.org/plotting-histogram-in-python-using-matplotlib/
# https://stackoverflow.com/questions/20128898/get-data-points-from-a-histogram-in-python
import seaborn as sns

# Iterate through each entry of sentiment dictionary
for key, value in sentiment_dict.items():
    # Plot a histogram of sentiment values, with 10 bins
    sns.histplot(value, bins=10, kde=True, color='lightgreen', edgecolor='red')
    # Adding labels and title
    plt.xlabel('Sentiment')
    plt.ylabel('Frequency')
    plt.title(f'Sentiment Histogram for category {key}')
     
    # Display the plot
    plt.show()

In [ ]:
# Now iterate through each pair of categories, and find euclidian distance
# https://www.geeksforgeeks.org/calculate-the-euclidean-distance-using-numpy/
# https://stackoverflow.com/questions/9314576/calculate-distance-between-two-vectors-of-different-length
# First, we must fill the shorter lists with 0s

# Find largest list size
N = 0
for key, value in sentiment_dict.items():
    # If the new category is larger than current largest, update N
    if len(value) > N:
        N = len(value)

# Now fill the smaller lists with 0s
# https://stackoverflow.com/questions/43336837/making-equal-size-lists-in-python
for key, value in sentiment_dict.items(): 
    # If the category is smaller than the largest one,
    if len(value) < N:
        # Fill the end of the list with 0s
        value.extend([0] * (N-len(value)))

# Make list of all categories 
categories = ['DEG', 'NDG', 'HOM', 'APR', 'CMP']
# Iterate through each pair of categories
for i in range(0, len(sentiment_dict.keys())):
    for j in range(i + 1, len(sentiment_dict.keys())):
        # Assign categ 1 and categ 2
        categ_1 = categories[i]
        categ_2 = categories[j]
        # Assigns sentiment list 1 and 2
        sent_1 = sentiment_dict[categ_1]
        sent_2 = sentiment_dict[categ_2]
        # Find the Euclidian distance between these two lists
        dist = np.linalg.norm(np.array(sent_1) - np.array(sent_2))
        # Neatly print the distance to a degree of five decimal points
        print(f"Euclidian distance between {categ_1} and {categ_2}: {dist:.5f}")

## Task 9

In [ ]:
# Removing the stopwords from the tokens since we dont need those for empath categorization
tokens_by_category_stopwords_removed = {}

for index, row in tokens_by_category.iterrows():
    category = str(row['Category'])
    tokens = row['Tokens']

    tokens_by_category_stopwords_removed[category] = [word for word in tokens if word.lower() not in Stopwords]

# For some reason there was annoying NaN key in the dictionary, so this removes it
for key in tokens_by_category_stopwords_removed.keys():
    if pd.isna(key):
        tokens_by_category_stopwords_removed.pop(key)
        break

In [ ]:
# Creating a vocabulary for each category for the embedding vectors
vocabularies_by_category_stopwords_removed = {}

for category, tokens in tokens_by_category_stopwords_removed.items():
    vocabularies_by_category_stopwords_removed[category] = set(tokens)

In [ ]:
# Creating the embedding vectors for each category with Empath
from empath import Empath
lexicon = Empath()

embedding_vectors_by_category = {}

for category, vocabulary in vocabularies_by_category_stopwords_removed.items():
    vocabulary_string = ' '.join(vocabulary)
    embedding_vector = lexicon.analyze(vocabulary_string, normalize=True)
    embedding_vectors_by_category[category] = embedding_vector

# annoying NaN key came back again
if 'nan' in embedding_vectors_by_category:
    del embedding_vectors_by_category['nan']

In [ ]:
# compute cosine similarity between the embedding vectors of each pair of categories
from sklearn.metrics.pairwise import cosine_similarity

categories = ['DEG', 'NDG', 'HOM', 'APR', 'CMP']

empath_cosine_similarities = {}

for i in range(0, len(categories)):
    for j in range(i + 1, len(categories)):
        category1 = categories[i]
        category2 = categories[j]

        vector1 = list(embedding_vectors_by_category[category1].values())
        vector2 = list(embedding_vectors_by_category[category2].values())
        cosine_sim = cosine_similarity([vector1], [vector2])[0][0]
        empath_cosine_similarities[(category1, category2)] = cosine_sim

In [ ]:
print("Cosine similarities between the embedding vectors of each pair of categories with Empath:")
for pair in empath_cosine_similarities:
    print(f"{pair[0]} and {pair[1]}: {empath_cosine_similarities[pair]:.2f}")
print("This way comparing the categories fails to capture where the categories are similar and where they are not.")
print("The tool gives a score for many different empath categories,")
print("and when the comparison between categories is combined into one number, information gets lost.")

## Task 10

In [ ]:
# from gensim.models import Doc2Vec

# # Load the model
# model_path = 'models/doc2vec.bin'
# model = Doc2Vec.load(model_path)

# # Calculate embeddings for each category
# category_embeddings = {}

# for category, tokens in tokens_by_category_stopwords_removed.items():
#     token_vectors = [model.infer_vector(token) for token in tokens if token in model.wv]
#     if token_vectors:
#         category_embeddings[category] = np.mean(token_vectors, axis=0)
#     else:
#         category_embeddings[category] = np.zeros(model.vector_size)

# # Calculate the cosine similarities
# similarity_df = pd.DataFrame(index=category_embeddings.keys(), columns=category_embeddings.keys())

# for category1, embedding1 in category_embeddings.items():
#     for category2, embedding2 in category_embeddings.items():
#         similarity = cosine_similarity([embedding1], [embedding2])[0][0]
#         similarity_df.loc[category1, category2] = similarity

# print(similarity_df)

In [ ]:
for index, row in bodies_by_category.iterrows():
    bodies, label = row['Bodies'], row['Category']

In [ ]:
# Use DistilBERT to compute the embeddings of each post in a category and average them to get the category embedding
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import random

# Load the pre-trained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

if torch.cuda.is_available():
    print("CUDA is available! You can use your GPU.")
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Running on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print number of bodies in each category
for category, bodies in bodies_by_category.iterrows():
    print(f'{category} category number of bodies: {len(bodies[1])}')

# Function to get the embeddings of a sentence
def get_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.cpu()

# Calculate the embeddings for each post in a category and average them to get the category embedding
category_embeddings = {}

for index, row in bodies_by_category.iterrows():
    
    bodies, category = row['Bodies'], row['Category']
    bodies = random.sample(bodies, 100)  # Sample 100 random posts from each category
    
    embeddings_sum = torch.zeros(model.config.dim)
    num_valid_bodies = 0  # Keep track of valid (non-empty) posts

    for body in bodies:
        if body.strip():  # Check if the body has content
            body_embedding = get_embeddings(body)
            embeddings_sum += body_embedding
            num_valid_bodies += 1

    if num_valid_bodies > 0:
        category_embeddings[category] = embeddings_sum / num_valid_bodies
    else:
        category_embeddings[category] = torch.zeros(model.config.dim)

# Print the average embeddings for each category
for category, embedding in category_embeddings.items():
    # get the category name from the index
    print(f'Category name: {category}')
    print(f'  Mean: {embedding.mean().item():.4f}')
    print(f'  Min: {embedding.min().item():.4f}')
    print(f'  Max: {embedding.max().item():.4f}')
    
# Calculate the cosine similarities of each pair of categories
cosine_similarities = {}

for category1, embedding1 in category_embeddings.items():
    for category2, embedding2 in category_embeddings.items():
        similarity = torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0).item()
        cosine_similarities[(category1, category2)] = similarity
        
# Print the cosine similarities
for categories, similarity in cosine_similarities.items():
    print(f'{categories[0]} and {categories[1]}: {similarity:.4f}')



## Task 11

In [ ]:
"""We want to evaluate the various categories in terms of linguistic quality of the posts. For this purpose,
suggest a script to identify the number of unmatched tokens in each dataframe, for instance by seeking
whether the token has an entry in Wordnet. You may also inspire by some existing implementation such
as Spelling Correction Of The Text Data In Natural Language Processing | by Nutan | Medium. Estimate
the number of spelling errors in each category and its proportion with respect to the total number of
posts in the category."""

In [ ]:
# Make function to run through all bodies of a specified category
# Create dictionary to hold how many errors are in each category
# https://medium.com/@nutanbhogendrasharma/spelling-correction-of-the-text-data-in-natural-language-processing-e9848407cf3b

# Import spell checker and initiate it
from spellchecker import SpellChecker
spell = SpellChecker()

# Make a dictionary to track errors in each category
error_dict ={"DEG": 0,
             "NDG": 0,
             "HOM": 0,
             "APR": 0,
             "CMP": 0
             }

# Make a dictionary to track the number of posts in each dictionary, this will be useful later
posts_dict ={"DEG": 0,
             "NDG": 0,
             "HOM": 0,
             "APR": 0,
             "CMP": 0
             }
# Make function to analyze posts of quality under specific category
def quality_analysis(categ):
    # Iterate each index
    for index, row in data.iterrows():
        # Create body and label integer and string
        body, label = str(row['body']), str(row['gold_label'])
        # If the label is the one we are looking for, continue
        if label == categ_string_list[categ]:
            # Print unknown, mispelt words
            # Remove punctuation as it seemingly affects the spellchecker
            #print(spell.unknown(body.split()))
            potential_error = spell.unknown(tokenizer.tokenize(body))
            # Update dictionary values
            error_dict[categ_string_list[categ]] += len(potential_error)
            posts_dict[categ_string_list[categ]] += 1
quality_analysis(0)
quality_analysis(1)
quality_analysis(2)
quality_analysis(3)
quality_analysis(4)

In [ ]:
# See the proportion of errors per post for each category
for key, value in error_dict.items():
    print(f"                 Potential errors in categ {key}: {value}")
    print(f"Proportion of errors per post for category {key}: {value/posts_dict[key]}")

In [ ]:
"""Here we can see the cateogry with the most potential spelling erros per post is CMP, and the category with the 
least potential spelling erros per post is DEG. This data may not be the best representation since there are far more
posts in DEG than CMP."""

## Task 12

In [ ]:
from spellchecker import SpellChecker
from nltk.metrics import edit_distance
import os

spell = SpellChecker()

already_found = {}

# Function to find the closest correction and calculate edit distance
def find_corrections_and_distances(categ):
    corrections = []
    distances = []

    tokens = tokens_by_category_stopwords_removed[categ_string_list[categ]]
    
    misspelled = spell.unknown(tokens)
    
    for word in misspelled:
        if word in already_found.keys():
            correction = already_found[word]
        else:
            correction = spell.correction(word)
            if correction:
                already_found[word] = correction
        if correction:
            corrections.append((word, correction))
            distances.append(edit_distance(word, correction))

    print(f"Category: {categ_string_list[categ]}")
    
    return corrections, distances

# Dictionary to store corrections and distances for each category
corrections_dict = {}
distances_dict = {}

# Get corrections and distances for each category from files if they exist
if os.path.exists('corrections.csv') and os.path.exists('distances.csv'):
    with open('corrections.csv', 'r', encoding='utf-8') as f:
        current_category = None
        for line in f:
            line = line.strip()
            if ',' not in line:
                current_category = line
                corrections_dict[current_category] = []
            else:
                correction = line.split(',')
                corrections_dict[current_category].append((correction[0], correction[1]))
    with open('distances.csv', 'r', encoding='utf-8') as f:
        current_category = None
        for line in f:
            line = line.strip()
            if any(char.isalpha() for char in line):
                current_category = line
                distances_dict[current_category] = []
            else:
                distances_dict[current_category].append(int(line))
else:
    # Find corrections and distances for each category
    for i in range(len(categ_string_list)-1):
        corrections, distances = find_corrections_and_distances(i)
        corrections_dict[categ_string_list[i]] = corrections
        distances_dict[categ_string_list[i]] = distances

In [ ]:
# Save the corrections and distances to a file
if not os.path.exists('corrections.csv') and not os.path.exists('distances.csv'):
    with open('corrections.csv', 'w', encoding='utf-8') as f:
        for category, corrections in corrections_dict.items():
            f.write(f'{category}\n')
            for correction in corrections:
                f.write(f'{correction[0]},{correction[1]}\n')
    with open('distances.csv', 'w', encoding='utf-8') as f:
        for category, distances in distances_dict.items():
            f.write(f'{category}\n')
            for distance in distances:
                f.write(f'{distance}\n')

In [ ]:
# Analyze edit distances by category by plotting a histogram
import seaborn as sns
import matplotlib.pyplot as plt

for category, distances in distances_dict.items():
    p = sns.histplot(distances, bins=range(0, max(distances) + 2), kde=False)
    
    p.set_xticks(range(0, max(distances) + 2))
    
    # Add labels and title
    plt.xlabel('Edit Distance')
    plt.ylabel('Frequency')
    plt.title(f'Edit Distance Histogram for category {category}')
    
    # Add gridlines to improve readability
    plt.grid(visible=True, which='major', linestyle='--', linewidth=0.5)
    
    # Show the plot
    plt.show()

In [ ]:
import random

# Save corrections to txt file
with open('corrections.txt', 'w', encoding='utf-8') as f:
    for category, corrections in corrections_dict.items():
        f.write(f'Category: {category}\n')
        for correction in corrections:
            f.write(f'{correction[0]} -> {correction[1]}\n')
        f.write('\n')

# Print table of results
print("Edit distances by category:")
for category, distances in distances_dict.items():
    print(f"{category}:")
    print(f"Mean: {np.mean(distances):.2f}")
    print(f"Median: {np.median(distances)}")
    print(f"Standard deviation: {np.std(distances):.2f}")
    print()